In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import matplotlib

In [ ]:
import os
import struct
import numpy as np
from utils import *



In [ ]:
import sklearn
from sklearn.manifold import TSNE
from time import time

In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [ ]:
from glow import thops
from glow import modules
from glow import models
from glow.config import JsonConfig
import cv2

In [ ]:
# negative
path_total = '/home/hyshuai/workspace/dataset/images_background'

dataset_negative = []
for path_p in os.listdir(path_total):
        path_p_join = os.path.join(path_total,path_p)
        for path_c in os.listdir(path_p_join):
            path_real = os.path.join(path_p_join,path_c)
            class_name = path_real
            # image_paths = get_image_paths(path_real)
            images = get_images_frompath(path_real)
            print(class_name," : ",len(images))
            dataset_negative.append(ImageObject(class_name,images))

In [ ]:
# positive

path_positive_total = '/home/hyshuai/workspace/dataset/images_evaluation'
class_name_list = []
for path_p in os.listdir(path_positive_total):
    path_p_join = os.path.join(path_positive_total,path_p)
    for path_c in os.listdir(path_p_join):
        path_real = os.path.join(path_p_join,path_c)
        class_name = path_real
        class_name_list.append(str(class_name))

# class_name = np.array(class_name)
# print(class_name_list)
np.random.seed(66) # 切换类试下
shufindex = np.random.permutation(len(class_name_list))
# print(shufindex)
class_positive = np.array(class_name_list)[shufindex][:20] #取5个类做测试 5-way
print(class_positive)

dataset_train_positive =[]
dataset_test_positive = []
for class_path in class_positive :
    images = get_images_frompath_random(class_path) # 固定随机抽样是否有问题
    print(class_path, " : ", len(images))
    dataset_train_positive.append(ImageObject(class_path, images[:1])) # 1-shot
    dataset_test_positive.append(ImageObject(class_path, images[1:]))



print(len(dataset_train_positive[2].imgs))
print(len(dataset_test_positive[2].imgs))






    

In [ ]:
# label:

class_total_num = len(dataset_negative) + len(dataset_train_positive)
print("class_total = ",class_total_num)
np.random.seed(66) 
shufindex = np.random.permutation(class_total_num)
for index, value in enumerate(dataset_negative):
    value.label = shufindex[index]
    print(value.name,  value.label)

for index, value in enumerate(dataset_train_positive):
    value.label = shufindex[index+len(dataset_negative)]
    dataset_test_positive[index].label = shufindex[index+len(dataset_negative)]
    print(value.name, value.label)

In [ ]:
# test_class_index = [122,51,631,828,20]
test_class_index = [data.label for data in dataset_train_positive]
print(test_class_index)

In [ ]:
# print(dataset_test_positive[0].name)
# np.random.shuffle(dataset_test_positive)
# print(dataset_test_positive[0].name)
# imgs_show(np.squeeze( dataset_test_positive[0].imgs),1,5) 
# imgs_show(np.squeeze(dataset_negative[1].imgs),4,5)


In [ ]:
# print(dataset_negative[0].imgs[2].shape)
# # print(im[15])
# # print(im.shape)
# print(dataset_negative[0].imgs[0].shape)
x = [data.imgs for data in dataset_train_positive] 
print(x[0][0].shape)
img_show(np.squeeze(x[0]))
img_show(np.squeeze(x[1]))
img_show(np.squeeze(x[2]))
img_show(np.squeeze(x[3]))
img_show(np.squeeze(x[4]))

In [ ]:

def get_next_perfect(dataset_train_positive,dataset_negative,p_size,n_size):
    """
    final batch_size = p_size + n_size * 2
    every dataset_negative should have same imgs leng
    """
    #shuffle
    np.random.shuffle(dataset_train_positive)
    np.random.shuffle(dataset_negative)
    for data in dataset_negative:
        np.random.shuffle(data.imgs)
    for data in dataset_train_positive:
        np.random.shuffle(data.imgs)
    
    
    batch_negative = n_size * 2
#     print('batch_negative ',batch_negative)
    len_n_class = len(dataset_negative)
#     print('len_n_class ',len_n_class)
    len_p_class = len(dataset_train_positive)
#     print('len_p_class ',len_p_class)
    len_n_imgs = len(dataset_negative[0].imgs)
#     print('len_n_imgs ',len_n_imgs)
    len_p_imgs = len(dataset_train_positive[0].imgs)
    
    negative_number = 0
    
    
#     shuffindex = np.random.permutation(len_n)
# #     print(shuffindex)
#     np.random.shuffle(dataset_train_positive)
#     np.random.shuffle(dataset_negative)
    
    for data in dataset_negative:
        negative_number += len(data.imgs)
#         np.random.shuffle(data.imgs)
#     print('negative_number ',negative_number)
    batch_total = negative_number  // batch_negative
#     print('batch_total ',batch_total)
    i_n_class = 0
    i_n_img = 0
    i_p_class = 0
    i_p_img = 0
    
    
    
        
    
    for b in range(batch_total):
        train_x =[]
        train_y = []
        create_negative_imgs = 0
        while create_negative_imgs <= (batch_total * batch_negative) and create_negative_imgs < batch_negative:
            
            # while #如果一次循环没够，需要再次循环
            if i_n_class >= len_n_class:
                    i_n_class = 0
                    i_n_img +=1
                    if (i_n_img+1) *2 >len_n_imgs:
                        i_n_img = 0
#                     print('if i_n_class >= len_n_class i_n_class ',i_n_class)
#                     print('if i_n_class >= len_n_class i_n_img ',i_n_img)
            
            for i in range(i_n_class,len_n_class):
                    
#                 print('for i in range(i_n_class i_n_class ',i_n_class)
#                 print('for i in range(i_n_class i_n_img ',i_n_img)
                train_x += dataset_negative[i_n_class].imgs[i_n_img * 2 : (i_n_img+1) *2]
                train_y += [dataset_negative[i_n_class].label] * 2
                
                i_n_class+=1
                if i_n_class >= len_n_class:
                    i_n_class = 0
                    i_n_img +=1
                    if (i_n_img+1) *2 > len_n_imgs:
                        i_n_img = 0
#                     print('if i_n_class >= len_n_class i_n_class ',i_n_class)
#                     print('if i_n_class >= len_n_class i_n_img ',i_n_img)
            
                create_negative_imgs +=2
                if create_negative_imgs >= batch_negative:
#                     print("create_negative_imgs ok ",create_negative_imgs)
                    break
                    
                
        create_positive_imgs = 0    
        while create_positive_imgs < p_size:
            for i in range(i_p_class,len_p_class):
#                 print('for i in range(i_p_class     ',i_p_class)
#                 print('for i in range(i_p_img  ',i_p_img)
                
                train_x += dataset_train_positive[i_p_class].imgs[i_p_img:(i_p_img +1)]
                train_y += [dataset_train_positive[i_p_class].label]
                
                i_p_class+=1
                if i_p_class >= len_p_class:
                    i_p_class = 0
                    i_p_img +=1
                    if (i_p_img +1) > len_p_imgs:
                        i_p_img = 0
#                     print('if i_p_class >= len_p_class i_p_class ',i_p_class)
#                     print('if i_p_class >= len_p_class i_p_img ',i_p_img)
                
                create_positive_imgs+=1
                if create_positive_imgs >= p_size:
#                     print("create_positive_imgs ok ",create_positive_imgs)
                    break
        
        shuffindex_out = np.random.permutation(len(train_y))
#         print(train_x[0].shape)
        train_x = np.stack(train_x,axis = 0)
#         print('after concat : ',train_x[0].shape)
        train_x = train_x[shuffindex_out]
        train_y = np.array(train_y)[shuffindex_out]
        
        yield train_x,train_y  

In [ ]:
# dataaaaaaa = get_next_perfect(dataset_train_positive,dataset_negative,8,126)
# for x ,y in dataaaaaaa:
#     print(y)
#     print('x o shape ' ,x.shape,y.shape)
#     print("one batch is over")


In [ ]:
def getnext_test(dataset_test_positive,test_class_index):
    
    for data in dataset_test_positive:
        x = np.stack(data.imgs,axis =0)
        y = np.array([test_class_index.index(data.label)] * len(data.imgs))
        yield x,y

In [ ]:
# dataaaaaaa = getnext_test(dataset_test_positive,test_class_index)
# for x ,y in dataaaaaaa:
#     print(y)
#     print(x.shape,y.shape)

In [ ]:
def imgs_show_norm(imgs,row,col):
    fig,ax = plt.subplots(nrows=row,ncols=col,sharex=True,sharey=True)
    ax = ax.flatten()
    for i in range(row*col):
        img = imgs[i]
        min = np.min(img)
#         print('min = ',min)
        img = np.subtract(img,min) # 0->
        max = np.max(img)
#         print('max =',max)
        img = np.divide(img,max)
        
        ax[i].imshow(img,cmap='Greys', interpolation='nearest')
    
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    plt.tight_layout()
    plt.show()

In [ ]:
def img_show_norm(img):
    min_ = np.min(img)
#     print('min = ',min_)
    img = np.subtract(img,min_) # 0->
    max_ = np.amax(a = img,keepdims=False)
#     print('max =',max_)
    img = np.divide(img,max_)  # 0->1
#     print(img)
    plt.figure(figsize=(1,1))
    plt.imshow(img,cmap='Greys',interpolation='nearest')

In [ ]:
def img_show(img):
    plt.figure(figsize=(1,1))
    plt.imshow(img,cmap='Greys',interpolation='nearest')
    

In [ ]:
def imgs_show(imgs,row,col):
    fig,ax = plt.subplots(nrows=row,ncols=col,sharex=True,sharey=True)
    ax = ax.flatten()
    for i in range(row*col):
        img = imgs[i]
        
        ax[i].imshow(img,cmap='Greys', interpolation='nearest')
    
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    plt.tight_layout()
    plt.show()
    
    

In [ ]:
def compute_index_logical_equal(inputs,indexs):
    logic_result = (inputs==indexs[0])
    for value in indexs:
        logic_result = np.logical_or(inputs == value,logic_result)
    return logic_result


In [ ]:
# nb_digits = class_total_num
net_auto = models.Glow(JsonConfig("hparams/omni_all_bg.json"),test_class_index,is_mean=True,K=8,y_classes=class_total_num,is_split_2d=False)


In [ ]:
# for param in net_auto.parameters():
#     print(param.data.size())
net_auto.load_state_dict(torch.load('model_glow_ominst_all_bg_8k_20_way_778.pt'),strict=False)
    

In [ ]:

# net_auto = torch.load('model_autoEncode5.pkl')
# net_auto.load_state_dict(torch.load('model_glow_ominst19000.pt'))
# net_auto.load_state_dict(torch.load('model_glow_ominst_no_split2d_4k_4316.pt'))
# net_auto.load_state_dict(torch.load('model_glow_ominst_no_split2d_8k_9000.pt'))
use_cuda = torch.cuda.is_available()
torch.manual_seed(666)
device = torch.device("cuda" if use_cuda else "cpu")
net_auto.to(device)
net_auto.float()


criterion = nn.MSELoss()
optimizer = optim.Adam(net_auto.parameters(),lr=0.0001,weight_decay=1e-5) # decoder don't weight_decay


In [ ]:
#X_train_nor
batch_size = 512
net_auto.train()
for epoch in range(30001):
    #train:
    
    datas =  get_next_perfect(dataset_train_positive,dataset_negative,12,250)
#     datas = getnext(dataset_train_positive,dataset_negative)
    
    trainloss_g = 0
    trainloss_c = 0
    
    for x,y in datas:
        inputs ,lables = torch.from_numpy(x).float().to(device),\
            torch.from_numpy(y).long().to(device)
#         print(torch.sum(lables)) # = 1167
        
        y_onehot = torch.FloatTensor(batch_size, class_total_num).to(device)
        y_onehot.zero_()
        y_onehot.scatter_(1, lables.view(-1,1), 1)
#         print('y_onehot:', y_onehot)


       

        optimizer.zero_grad()
#         z, det, y_logits = net_auto(x=inputs, y_onehot=y_onehot)
        
        z, det = net_auto(x=inputs, y_onehot=y_onehot)
        classify = net_auto.class_flow(z)
        
#         print(z.size())
#         print(det)
        loss_g = models.Glow.loss_generative(det)
        loss_c = models.Glow.loss_class(y_logits,lables)
        print('loss_g = ',loss_g.item())
        print('loss_c =  ',loss_c.item())
        loss = loss_g + loss_c * 0.2
        loss.backward()
        torch.nn.utils.clip_grad_value_(net_auto.parameters(), 5)
        torch.nn.utils.clip_grad_norm_(net_auto.parameters(), 100)
        
        
        trainloss_g += loss_g.item()
        trainloss_c += loss_c.item()
#         print("net_auto.conv_d1 grad:",net_auto.conv_d1.weight.grad)
        
#         print("after net_auto.conv_1 grad:",net_auto.conv1.weight.grad)
        optimizer.step()
        
    print('%d  gloss: %.3f, class loss:%.3f' % (epoch ,  trainloss_g,trainloss_c))
    
    if epoch % 200 == 0 and epoch >0:
        torch.save(net_auto.state_dict(),'model_glow_ominst_all_bg_8k_20_way_'+str(epoch)+'.pt')
    


In [ ]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(666)
device = torch.device("cuda" if use_cuda else "cpu")
net_auto.to(device)
net_auto.float()
net_auto.eval()

test_loss = 0
correct = 0
test_datas = getnext_test(dataset_test_positive,test_class_index)
for x ,y in test_datas:
        inputs, labels = Variable(torch.from_numpy(x).float()).cuda(), Variable(torch.from_numpy(y).long()).cuda()
        print(labels)
        
        z, det = net_auto(x=inputs, y_onehot=None)
        classify = net_auto.class_flow(z)
        
        
        
#         _,classify = net_auto(inputs)
        test_loss += F.nll_loss(classify, labels, size_average=False).item() # sum up batch loss
#         print(classify)
        pred = classify.max(1,keepdim=True)[1]
#         print(pred)
        
        correct += pred.eq(labels.view_as(pred)).sum().item()
#         print(correct)
print(correct)
print(correct * 1.0 / (19 * 20))
print(test_loss)

In [ ]:
x_backgroud = np.array([data.imgs[0] for data in dataset_negative])
label_backgroud = np.array([data.label for data in dataset_negative])
x_positive = np.array([data.imgs[0] for data in dataset_train_positive])
label_positive = np.array([data.label for data in dataset_train_positive])

images_train_total = np.concatenate((x_backgroud ,x_positive)) 
label_total = np.concatenate((label_backgroud , label_positive))
print(images_train_total.shape)
print(label_total)
print(shufindex)
imgs_show(np.squeeze(images_train_total[0:20]),4,5)

In [ ]:
print(label_positive)
imgs_show(np.squeeze(x_positive),4,5)

In [ ]:
batch_size = len(images_train_total)
# lables = np.array([index for index in range(class_total_num)])
lables = torch.from_numpy(label_total).long().to(device)
print("get_log_mean_var ,labels = ", lables)
y_onehot = torch.FloatTensor(batch_size, class_total_num).cuda()
y_onehot.zero_()
y_onehot.scatter_(1, lables.view(-1, 1), 1)

mean,var = net_auto.get_log_mean_var(batch_size,y_onehot)


In [ ]:
std = torch.exp(var.cpu()).detach().numpy()

In [ ]:


print(std[-70:-20,2,0,0])
print(std[-20:,2,0,0])

In [ ]:
mean_x = mean.cpu().detach().numpy()
# mean_y = np.ones_like(mean_x)
color = (mean_x + 1 ) * 150
print(mean_x[-200:-20,0,0,0])
print(mean_x[-20:,0,0,0])

In [ ]:
u_total = np.mean(mean_x,axis=0)
print('u_total =  ',u_total[:64,0,0])
std_mean = np.mean(np.square(std),axis=0)
print('std_mean =  ',std_mean[:10,0,0])
ui_squ_mean = np.mean(np.square(mean_x),axis=0)
print('ui_squ_mean =  ',ui_squ_mean[:10,0,0])
u_total_sque = np.square(u_total)
print('u_total_sque =  ',u_total_sque[:10,0,0])
std_squ_total = std_mean + ui_squ_mean - u_total_sque

print('std_squ_total =  ',std_squ_total[:10,0,0])
print('std__total = ',np.sqrt(std_squ_total)[:,0,0])

In [ ]:
plt.figure(figsize=(200,100))
plt.scatter(x = mean_x[:,0,0,0],y = np.zeros_like(mean_x[:,0,0,0]),s=75,c =color[:,0,0,0],  alpha=.5)
plt.show()

In [ ]:
criterion = nn.MSELoss(reduction='sum')
ones_tensor = torch.ones((1, 32, 32)).float().cuda()

from scipy import misc
import time

In [ ]:
# net_auto.load_state_dict(torch.load('model_glow_ominst2000.pt'))


# use_cuda = torch.cuda.is_available()
# torch.manual_seed(666)
# device = torch.device("cuda" if use_cuda else "cpu")
# net_auto.to(device)
# net_auto.float()
# net_auto.load_state_dict(torch.load('model_glow_ominst_no_split2d_no_z_mean_no_add_normal1223.pt'))

# net_auto.load_state_dict(torch.load('model_glow_ominst_no_split2d_8k_9000.pt'))
# net_auto.train()
# torch.save(net_auto.state_dict(),'model_glow_ominst_all_bg_8k_20_way_778.pt')

def sample_and_save(sample_p,p,sampe_index_num):
    
    sample_n = np.random.randint(964)
#     sample_index = np.random.randint(18)
    sample_index = np.random.permutation(20)[:2]
    
    print('sample_p = {},sample_n ={}, sample_index = {}'.format(sample_p,sample_n,sample_index))

    posi = np.concatenate(( np.array(dataset_train_positive[sample_p].imgs)  ,  np.array(dataset_negative[sample_n].imgs)[sample_index]   ))
    labl =  np.array([dataset_train_positive[sample_p].label, dataset_negative[sample_n].label,dataset_negative[sample_n].label])
    print(dataset_train_positive[sample_p].name)
    save_path = dataset_train_positive[sample_p].name.split('/home/hyshuai/workspace/dataset/images_evaluation/')[1]
    save_path = './generate_img/' + str(save_path)
    
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    print(save_path)
    
    inputs ,lables = torch.from_numpy(posi).float().to(device),\
                torch.from_numpy(labl).long().to(device)

    y_onehot = torch.FloatTensor(3, class_total_num).to(device)
    y_onehot.zero_()
    y_onehot.scatter_(1, lables.view(-1,1), 1)
    # print('y_onehot:', y_onehot)
    # z, det, y_logits = net_auto(x=inputs, y_onehot=y_onehot)
    z, det = net_auto(x=inputs, y_onehot=y_onehot)
    y_logits = net_auto.class_flow(z)
    
    
    obj = z[0] 
    nn_index = [-1,1]
#     print(torch.sum(torch.abs(z[2]),dim =(1,2)))
#     print(torch.sum(torch.abs(z[1]),dim =(1,2)))

#     print(torch.sum(torch.abs(z[0]),dim =(1,2)))
    #         p = np.random.normal(loc=0.5,scale=0.1)
    q = np.random.randint(0, 2)
    scale = nn_index[q] * p
    # print('scale = ', scale)
#     start_index = np.random.randint(0,44)
    # start_index = 0
#     end_index = start_index + 20

    slic_index = np.random.permutation(64)[:sampe_index_num]
#     print(slic_index) 

#     print('scale = {},start_index ={}, end_index = {}'.format(scale,start_index,end_index))
    # obj[start_index:end_index] = obj[start_index:end_index] + scale * (z[1][start_index:end_index] - z[2][start_index:end_index])
    obj[slic_index] = obj[slic_index] + scale * (z[1][slic_index] - z[2][slic_index])

    # print(obj.size())
    obj = obj.view(-1,*obj.size())
    # print(obj.size())
    obj_oneshot = y_onehot[0].view(-1,class_total_num)
    
    
    
    x_ = net_auto(z = obj,y_onehot = obj_oneshot,eps_std = None ,reverse=True)

    x_ = torch.clamp(x_,-1,1)
    
    diffrent_b = criterion(x_[0] , inputs[0]) / criterion(inputs[0],ones_tensor)
    print(' diffrent_b = ',diffrent_b)

    c_self = criterion(inputs[0],ones_tensor)
    
    x_ = np.squeeze(x_.data.cpu().numpy())
#     print(x_.shape)
    tt = int(round(time.time() * 1000))
    misc.imsave(save_path+"/"+str(tt)+"_"+str(diffrent_b.data.cpu())+ ".png",x_)
#     x_ = cv2.imwrite()
    

    



In [ ]:
for i in range(14,20):

    for _ in range(10000):

        sample_and_save(i,0.5,64)

In [ ]:
sample_p = 0 # np.random.randint(20)  # 10 # 
sample_n = np.random.randint(964)
sample_index = np.random.randint(18)
print('sample_p = {},sample_n ={}, sample_index = {}'.format(sample_p,sample_n,sample_index))

posi = np.concatenate(( np.array(dataset_train_positive[sample_p].imgs)  ,  np.array(dataset_negative[sample_n].imgs[sample_index:sample_index+2])   ))
labl =  np.array([dataset_train_positive[sample_p].label, dataset_negative[sample_n].label,dataset_negative[sample_n].label])
print(dataset_train_positive[sample_p].name)
inputs ,lables = torch.from_numpy(posi).float().to(device),\
            torch.from_numpy(labl).long().to(device)
        
y_onehot = torch.FloatTensor(3, class_total_num).to(device)
y_onehot.zero_()
y_onehot.scatter_(1, lables.view(-1,1), 1)
# print('y_onehot:', y_onehot)
# z, det, y_logits = net_auto(x=inputs, y_onehot=y_onehot)
z, det = net_auto(x=inputs, y_onehot=y_onehot)
y_logits = net_auto.class_flow(z)


print('z.size() ' ,z.size())
print('y_logits.size() ' ,y_logits.size())
print(y_logits[1][1])

In [ ]:
obj = z[0]
# print(obj[0])
obj = z[0] 
nn_index = [-1,1]
print(torch.sum(torch.abs(z[2]),dim =(1,2)))
print(torch.sum(torch.abs(z[1]),dim =(1,2)))

print(torch.sum(torch.abs(z[0]),dim =(1,2)))
p =  1.0 # np.random.uniform(0.5, 0.8)
#         p = np.random.normal(loc=0.5,scale=0.1)
q = np.random.randint(0, 2)
scale = nn_index[q] * p
# print('scale = ', scale)
start_index = np.random.randint(0,44)
# start_index = 0
end_index = start_index + 20

slic_index = np.random.permutation(64)[:20]
print(slic_index) 

print('scale = {},start_index ={}, end_index = {}'.format(scale,start_index,end_index))
# obj[start_index:end_index] = obj[start_index:end_index] + scale * (z[1][start_index:end_index] - z[2][start_index:end_index])
obj[slic_index] = obj[slic_index] + scale * (z[1][slic_index] - z[2][slic_index])

# print(obj.size())
obj = obj.view(-1,*obj.size())
# print(obj.size())
obj_oneshot = y_onehot[0].view(-1,class_total_num)
# print(obj_oneshot.size())
# print(obj_oneshot)
# print(obj[0][0])

In [ ]:
# z = z.detach().cpu().numpy()
# print(z.size())
# print(z[0][0][0])
x_ = net_auto(z = obj,y_onehot = obj_oneshot,eps_std = None ,reverse=True)

x_ = torch.clamp(x_,-1,1)
print(x_.size())

print("abs sum = ",torch.sum(torch.abs(x_[0] - inputs[0])))
# print("abs sum = ",torch.sum(x_[0]- inputs[0]))
c = criterion(x_[0] , inputs[0])
print("criterion sum   = ",c)

c_self = criterion(inputs[0],ones_tensor)
print("self criterion sum  = ",c_self)
dd =  torch.sum(torch.mul(x_[0],inputs[0]))
print("  sum  = ",  dd )
m1 = torch.norm(x_[0])
print("  sum  = ",  m1 )
m2 = torch.norm(inputs[0])
print("  sum  = ",  m2 )

# l2_self = torch.sum(np.square(x_[0] - inputs[0]))
# print("l2_self sum  = ",l2_self)

print("l2  bi li   = ",c / c_self)
print("cos distance   = ",dd  / (m1 * m2) )

In [ ]:
img_show(np.squeeze(x_.data))
img_show(np.squeeze(inputs.data[0]))
imgs_show(np.squeeze(inputs.data),1,3)

In [ ]:
train_img_list = [data.imgs for data in dataset_train_positive]
train_name_list = [data.name for data in dataset_train_positive]
imgs_show(np.squeeze(np.array(train_img_list)),4,5)
print(train_name_list)

In [ ]:
def img_show(img):
    plt.figure(figsize=(1,1))
    plt.imshow(img,cmap='Greys',interpolation='nearest')

In [ ]:
# import seaborn as sns
# sns.set_style('darkgrid')
# sns.set_palette('muted')
# sns.set_context("notebook", font_scale=1.5,
# rc={"lines.linewidth": 2.5})

label_class = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
tsne = TSNE(n_components=2, init='pca', random_state=501)
X = np.squeeze(np.array(train_img_list))

X = np.reshape(X,(-1,32 * 32))
print(X.shape)
result = tsne.fit_transform(X)

t0 = time()

fig = plot_embedding(result, test_class_index,'t-SNE embedding of the digits (time %.2fs)'% (time() - t0))
plt.show(fig)

In [ ]:
# plt.axis([0, 1000, 0, 1])
# plt.ion()
 
# while True:
#     x = []
#     y = []
#     for i in range(512):
#         x.append(i)
#         y.append(np.random.random())
#         # plt.pause(0.05)
#     plt.cla()
#     plt.plot(x, y)
#     plt.pause(0.033)


In [ ]:
dataaaaaaa = get_next_perfect(dataset_train_positive,dataset_negative,8,126)

z_list = []
labels_list = []
for x ,y in dataaaaaaa:
    inputs, labels = Variable(torch.from_numpy(x).float()).cuda(), Variable(torch.from_numpy(y).long()).cuda()
    print(labels)
    z, det = net_auto(x=inputs, y_onehot=None)
    z_list.append(z.data.cpu())
    labels_list.append(y)
print(len(z_list))
print(len(labels_list))
    


In [ ]:
z_array = np.concatenate(z_list)
print(z_array.shape)
labels_array = np.concatenate(labels_list)
print(labels_array.shape)

In [ ]:
z_review = np.reshape(z_array,(-1,64 * 4 * 4))
print(z_review.shape)

In [ ]:
def plot_embedding(data, label, title):
    x_min, x_max = np.min(data, 0), np.max(data, 0)
    data = (data - x_min) / (x_max - x_min)

    fig = plt.figure(figsize=(50,50))
    ax = plt.subplot(111)
    for i in range(data.shape[0]):
        plt.text(data[i, 0], data[i, 1], str(label[i]),
                 color=plt.cm.Set1(label[i] / 989.),
                 fontdict={'weight': 'bold', 'size': 9})
    plt.xticks([])
    plt.yticks([])
    plt.title(title)
    return fig

In [ ]:
tsne = TSNE(n_components=2, init='pca', random_state=666)

result_z = tsne.fit_transform(z_review[:2000])

t0 = time()



In [ ]:
# print()
fig = plot_embedding(result_z, labels_array[:2000],'t-SNE embedding of the digits (time %.2fs)'% (time() - t0))
plt.show(fig)